In [1]:
import pdb
from nltk.corpus import stopwords, twitter_samples
import numpy as np
import pandas as pd
import nltk
import string
from nltk.tokenize import TweetTokenizer
from os import getcwd
from sklearn.model_selection import train_test_split
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
import json
import re
from nltk.corpus import wordnet
from collections import Counter
cnt = Counter()

In [2]:
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

In [3]:
## nltk.download('averaged_perceptron_tagger')

Reading Data

In [4]:
twitterData = pd.read_csv('twitter-sanders-apple2.csv')

In [5]:
twitterData

,class,text
0,Pos,Now all @Apple has to do is get swype on the i...
1,Pos,@Apple will be adding more carrier support to ...
2,Pos,Hilarious @youtube video - guy does a duet wit...
3,Pos,@RIM you made it too easy for me to switch to ...
4,Pos,I just realized that the reason I got into twi...
...,...,...
474,Neg,Houston we have a problem!! My iPad has been r...
475,Neg,#Siri went down for a little while last night....
476,Neg,@ford should have teamed up with @Apple instea...
477,Neg,RT @gdcurry: Really @Apple? What have you don...


Preprocess Data

In [6]:
twitterData["text"] = twitterData["text"].str.lower()
STOPWORDS = set(stopwords.words('english'))
STOPWORDS.remove('no')
STOPWORDS.remove('not')

In [7]:
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

stemmer = PorterStemmer()
def stem_words(text):
    return " ".join([stemmer.stem(word) for word in text.split()])


In [95]:
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def chatacronyms_to_text(text):
    with open('emoticon.json', 'r', encoding='utf-8') as json_file:
        chat_dict = json.load(json_file)
    for words in chat_dict:
        text = re.sub(rf'{words.lower()}',f'{chat_dict[words].lower()}',text)
    return text

def convert_emojis(text):
    with open('emoticon.json', 'r', encoding='utf-8') as json_file:
        UNICODE_EMO = json.load(json_file)
    for emot in UNICODE_EMO:
        text = re.sub(r'('+emot+')', "_".join(UNICODE_EMO[emot].replace(",","").replace(":","").split()), text)
    return text

def tag_words(text):
    wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}
    pos_tagged_text = nltk.pos_tag(text.split())
    # res  =[wordnet_map.get(pos[0]) for word, pos in pos_tagged_text]
    tagged_words = ['{}_{}'.format(word, pos) for word, pos in pos_tagged_text]
    print( ' '.join(tagged_words))

def check(text):
    wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}
    pos_tagged_text = nltk.pos_tag(text.split())
    tagged_words = ['{}_{}'.format(word, pos).lower() for word, pos in pos_tagged_text]
    return(tagged_words)

# Threshold =  3  
n_rare_words = 3
RAREWORDS = set([w for (w, wc) in cnt.most_common()[:-n_rare_words-1:-1]])
def remove_rarewords(text):
    """custom function to remove the rare words"""
    return " ".join([word for word in str(text).split() if word not in RAREWORDS])

In [107]:
res = []
for val in twitterData["features"].apply(lambda text: check(text)).to_list():
    res += val

res

['apple_nn',
 'get_vb',
 'swype_jj',
 'iphone_nn',
 'crack_nn',
 'iphone_nn',
 'apple_nn',
 'adding_vbg',
 'carrier_nn',
 'support_nn',
 'iphone_nn',
 '4_cd',
 'announced_vbd',
 'hilarious_jj',
 'youtube_nn',
 'video_nn',
 'guy_nn',
 'duet_nn',
 'apple_nn',
 'siri_nn',
 'pretty_rb',
 'much_jj',
 'sum_nn',
 'love_in',
 'affair_nn',
 'httptco8exbnqjy_nn',
 'rim_nn',
 'made_vbd',
 'easy_jj',
 'switch_nn',
 'apple_nn',
 'iphone_nn',
 'see_vbp',
 'ya_nn',
 'realized_vbn',
 'reason_nn',
 'got_vbd',
 'twitter_rb',
 'ios5_jj',
 'thanks_nns',
 'apple_nn',
 'im_jj',
 'current_jj',
 'blackberry_nn',
 'user_rb',
 'little_jj',
 'bit_nn',
 'disappointed_jj',
 'move_nn',
 'android_nn',
 'apple_nn',
 'iphone_nn',
 '16_cd',
 'strangest_jjs',
 'thing_nn',
 'siri_nn',
 'said_vbd',
 'far_rb',
 'ssurprisedo_jj',
 'glad_jj',
 'apple_nn',
 'gave_vbd',
 'siri_jj',
 'sense_nn',
 'humor_nn',
 'httptcotwaeudbp_nn',
 'via_in',
 'happyplace_nn',
 'great_jj',
 'close_jj',
 'personal_jj',
 'event_nn',
 'apple_nn',
 

In [9]:
twitterData["textcleant"] = twitterData["text"].apply(lambda text: remove_punctuation(text)).apply(lambda text:remove_stopwords(text))
twitterData["textcleant"] = twitterData["textcleant"].apply(lambda text: remove_urls(text)).apply(lambda text: chatacronyms_to_text(text))
twitterData["textcleant"] = twitterData["textcleant"].apply(lambda text : convert_emojis(text))
twitterData["features"] = twitterData["textcleant"].apply(lambda text: lemmatize_words(text))
twitterData["text_with_pos"] = twitterData["features"].apply(lambda text: tag_words(text))

In [61]:
twitterData["text_with_pos"] = twitterData["features"].apply(lambda text: tag_words(text))

In [28]:
twitterData

,class,text,textcleant,features,text_with_pos,binary_target
0,Pos,now all @apple has to do is get swype on the i...,apple get swype iphone crack iphone,apple get swype iphone crack iphone,apple_NN get_VB swype_JJ iphone_NN crack_NN ip...,1
1,Pos,@apple will be adding more carrier support to ...,apple adding carrier support iphone 4s announced,apple adding carrier support iphone 4 announced,apple_NN adding_VBG carrier_NN support_NN ipho...,1
2,Pos,hilarious @youtube video - guy does a duet wit...,hilarious youtube video guy duet apple siri pr...,hilarious youtube video guy duet apple siri pr...,hilarious_JJ youtube_NN video_NN guy_NN duet_N...,1
3,Pos,@rim you made it too easy for me to switch to ...,rim made easy switch apple iphone see ya,rim made easy switch apple iphone see ya,rim_NN made_VBD easy_JJ switch_NN apple_NN iph...,1
4,Pos,i just realized that the reason i got into twi...,realized reason got twitter ios5 thanks apple,realized reason got twitter ios5 thanks apple,realized_VBN reason_NN got_VBD twitter_RB ios5...,1
...,...,...,...,...,...,...
474,Neg,houston we have a problem!! my ipad has been r...,houston problem ipad restoring 12 hours instal...,houston problem ipad restoring 12 hour install...,houston_NN problem_NN ipad_NN restoring_VBG 12...,0
475,Neg,#siri went down for a little while last night....,siri went little last night come apple get shi...,siri went little last night come apple get shi...,siri_JJ went_VBD little_JJ last_JJ night_NN co...,0
476,Neg,@ford should have teamed up with @apple instea...,ford teamed apple instead microsoft make sync ...,ford teamed apple instead microsoft make sync ...,ford_NN teamed_VBD apple_NN instead_RB microso...,0
477,Neg,rt @gdcurry: really @apple? what have you don...,rt gdcurry really apple done itunes cant click...,rt gdcurry really apple done itunes cant click...,rt_NN gdcurry_NN really_RB apple_NN done_VBN i...,0


In [11]:
label_map = {"Pos": 1, "Neg": 0}

# Convert target variable using map function
twitterData['binary_target'] = twitterData['class'].map(label_map)


In [12]:
twitterData

,class,text,textcleant,features,text_with_pos,binary_target
0,Pos,now all @apple has to do is get swype on the i...,apple get swype iphone crack iphone,apple get swype iphone crack iphone,apple_NN get_VB swype_JJ iphone_NN crack_NN ip...,1
1,Pos,@apple will be adding more carrier support to ...,apple adding carrier support iphone 4s announced,apple adding carrier support iphone 4 announced,apple_NN adding_VBG carrier_NN support_NN ipho...,1
2,Pos,hilarious @youtube video - guy does a duet wit...,hilarious youtube video guy duet apple siri pr...,hilarious youtube video guy duet apple siri pr...,hilarious_JJ youtube_NN video_NN guy_NN duet_N...,1
3,Pos,@rim you made it too easy for me to switch to ...,rim made easy switch apple iphone see ya,rim made easy switch apple iphone see ya,rim_NN made_VBD easy_JJ switch_NN apple_NN iph...,1
4,Pos,i just realized that the reason i got into twi...,realized reason got twitter ios5 thanks apple,realized reason got twitter ios5 thanks apple,realized_VBN reason_NN got_VBD twitter_RB ios5...,1
...,...,...,...,...,...,...
474,Neg,houston we have a problem!! my ipad has been r...,houston problem ipad restoring 12 hours instal...,houston problem ipad restoring 12 hour install...,houston_NN problem_NN ipad_NN restoring_VBG 12...,0
475,Neg,#siri went down for a little while last night....,siri went little last night come apple get shi...,siri went little last night come apple get shi...,siri_JJ went_VBD little_JJ last_JJ night_NN co...,0
476,Neg,@ford should have teamed up with @apple instea...,ford teamed apple instead microsoft make sync ...,ford teamed apple instead microsoft make sync ...,ford_NN teamed_VBD apple_NN instead_RB microso...,0
477,Neg,rt @gdcurry: really @apple? what have you don...,rt gdcurry really apple done itunes cant click...,rt gdcurry really apple done itunes cant click...,rt_NN gdcurry_NN really_RB apple_NN done_VBN i...,0


Q1) Vectorize the words(unigrams and bigrams boths) to corresponding vectors

In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(lowercase=True, min_df=3,ngram_range=(1,2))

tfidf_X = tfidf_vectorizer.fit_transform(twitterData['features'])


In [39]:
feature_names = tfidf_vectorizer.get_feature_names_out()

print("Feature names (unigrams and bigrams):")
print(feature_names)

Feature names (unigrams and bigrams):
['12' '25' '3gs' 'absolutely' 'actually' 'actually help' 'air' 'album'
 'already' 'always' 'amazing' 'android' 'annoyed' 'another' 'anyone'
 'anyone else' 'app' 'apparently' 'apparently apple' 'apple' 'apple apple'
 'apple battery' 'apple customer' 'apple dont' 'apple fix' 'apple fuck'
 'apple genius' 'apple get' 'apple icloud' 'apple io' 'apple ios5'
 'apple iphone' 'apple iphone4s' 'apple iphones' 'apple new' 'apple no'
 'apple not' 'apple please' 'apple product' 'apple put' 'apple sell'
 'apple siri' 'apple store' 'apple tech' 'apple thank' 'apple thanks'
 'apple umber' 'apple update' 'apple wont' 'apps' 'apps apple' 'as' 'att'
 'att apple' 'awesome' 'awesome little' 'awful' 'back' 'bad' 'bar'
 'battery' 'battery drain' 'battery life' 'battery new' 'best' 'better'
 'bgr' 'blackberry' 'blowing' 'blowing raspberryerience' 'bottom'
 'bottom see' 'bring' 'bsurprisedk' 'bug' 'button' 'buy' 'call' 'camera'
 'cannot' 'cant' 'cant connect' 'card' 'champ

Split the data into training and test data using Stratified Split [ To maintain the ratio of classes]

In [15]:
# Split features and target variable
X = tfidf_X
y = twitterData['binary_target']

# Initialize StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

# Get train and test indices
train_index, test_index = next(sss.split(X, y))

# Create stratified train and test datasets
X_train = X[train_index]
y_train = y[train_index]
X_test = X[test_index]
y_test = y[test_index]

# Print shapes of train and test data
print("Train data shape:", X_train.shape)
print("Test data shape:", X_test.shape)

Train data shape: (383, 492)
Test data shape: (96, 492)


In [119]:
X

<479x485 sparse matrix of type '<class 'numpy.float64'>'
	with 3695 stored elements in Compressed Sparse Row format>

Multinomial Naive Bayes for textual Data

In [16]:
clf = MultinomialNB()

# Train the classifier
clf.fit(X_train, y_train)

MultinomialNB()

In [17]:
from sklearn.metrics import classification_report
y_pred = clf.predict(X_test)
report = classification_report(y_test, y_pred)
print("Classification Report:")
print(report)

Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.98      0.87        63
           1       0.94      0.48      0.64        33

    accuracy                           0.81        96
   macro avg       0.86      0.73      0.76        96
weighted avg       0.84      0.81      0.79        96



In [124]:
X_test[0],y_pred,y_test

(<1x485 sparse matrix of type '<class 'numpy.float64'>'
 	with 5 stored elements in Compressed Sparse Row format>,
 array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1,
        0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 0], dtype=int64),
 115    1
 212    0
 36     1
 94     1
 323    0
       ..
 66     1
 268    0
 243    0
 176    0
 184    0
 Name: binary_target, Length: 96, dtype: int64)

In [ ]:
clf.predict('Apple is bad')

SVM with linear kernel

In [18]:
svm_clf = SVC(kernel='linear')
svm_clf.fit(X_train, y_train)

SVC(kernel='linear')

In [19]:
y_pred = svm_clf.predict(X_test)

report = classification_report(y_test, y_pred)

print("Classification Report:")
print(report)

Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.92      0.86        63
           1       0.79      0.58      0.67        33

    accuracy                           0.80        96
   macro avg       0.80      0.75      0.76        96
weighted avg       0.80      0.80      0.79        96



Naive Bayes (NB) is an extremely quick approach. It relies on conditional probabilities, which are simple to calculate and execute. Therefore, an iterative procedure is unnecessary. NB supports binary classification as well as multinomial one. NB implies that characteristics are independent of one another, however this is not necessarily the case. Even though, NB gives good results when applied to short texts like tweets. Using feature selection, NB may outperform other classifiers on certain datasets.

In the context of machine learning and statistics, the multinomial distribution is often used to model the distribution of categorical data, such as the counts of different categories in a sample or the outcomes of categorical experiments.

SVM is more effective for non-linear classification problems. SVM generalizes well in high-dimensional domains, such as those representing texts. It is effective with a greater number of dimensions than samples.

Q2) Vectorize the POS words(unigrams and bigrams boths) to corresponding vectors and compare with the above

In [158]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(lowercase=True, min_df=3,ngram_range=(1,2))

tfidf_X_pos = tfidf_vectorizer.fit_transform(twitterData['text_with_pos'])


In [159]:
type(tfidf_X_pos)

scipy.sparse._csr.csr_matrix

In [161]:
feature_names = tfidf_vectorizer.get_feature_names_out()

print("Feature names (unigrams and bigrams):")
print(feature_names)
print(tfidf_vectorizer.token_pattern)
tfidf_df = pd.DataFrame(tfidf_X_pos.toarray(), columns=tfidf_vectorizer.get_feature_names_out())



Feature names (unigrams and bigrams):
['12_cd' '1_cd' '25_cd' '2_cd' '3_cd' '3_cd day_nn' '3gs_cd' '4_cd'
 '4_cd million_cd' '5_cd' '9_cd' '_jj' '_jj cheeky' '_nn' '_nn playful_nn'
 'a_dt' 'a_dt raspberryerience_nn' 'absolutely_rb' 'actually_rb' 'air_nn'
 'album_nn' 'already_rb' 'always_rb' 'amazing_vbg' 'android_nn'
 'another_dt' 'anyone_nn' 'anyone_nn else_rb' 'app_nn' 'apparently_rb'
 'apparently_rb apple_nn' 'apple_nn' 'apple_nn apple_nn'
 'apple_nn battery_nn' 'apple_nn customer_nn' 'apple_nn dont_nn'
 'apple_nn fix_nn' 'apple_nn genius_nn' 'apple_nn io_nn'
 'apple_nn ios5_nn' 'apple_nn iphone4s_nn' 'apple_nn iphone_nn'
 'apple_nn iphones_nns' 'apple_nn new_jj' 'apple_nn no_dt'
 'apple_nn not_rb' 'apple_nn please_nn' 'apple_nn product_nn'
 'apple_nn sell_vbp' 'apple_nn siri_nn' 'apple_nn store_nn'
 'apple_nn thank_nn' 'apple_nn thanks_nns' 'apple_nn umber_nnp'
 'apple_nn update_jj' 'apps_nn' 'apps_nn apple_nn' 'as_in' 'att_nn'
 'awesome_jj' 'awesome_jj little_jj' 'awful_jj' 'back_

In [162]:
len(tfidf_X_pos.toarray()[0])

485

Split the data into training and test data using Stratified Split [ To maintain the ratio of classes]

In [163]:
# Split features and target variable
X = tfidf_X_pos
y = twitterData['binary_target']

# Initialize StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

# Get train and test indices
train_index, test_index = next(sss.split(X, y))

# Create stratified train and test datasets
X_train = X[train_index]
y_train = y[train_index]
X_test = X[test_index]
y_test = y[test_index]

# Print shapes of train and test data
print("Train data shape:", X_train.shape)
print("Test data shape:", X_test.shape)

Train data shape: (383, 485)
Test data shape: (96, 485)


In [164]:
len(X_train.toarray()[0])

485

In [165]:
y_test.shape

(96,)

In [166]:
X_test[1]

<1x485 sparse matrix of type '<class 'numpy.float64'>'
	with 9 stored elements in Compressed Sparse Row format>

Multinomial Naive Bayes for textual Data

In [167]:
clf.fit(X_train,y_train)

MultinomialNB()

In [168]:
from sklearn.metrics import classification_report
y_pred = clf.predict(X_test)
report = classification_report(y_test, y_pred)
print("Classification Report:")
print(report)

Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.97      0.87        63
           1       0.89      0.48      0.63        33

    accuracy                           0.80        96
   macro avg       0.84      0.73      0.75        96
weighted avg       0.82      0.80      0.78        96



SVM with linear kernel

In [25]:
svm_clf = SVC(kernel='linear')
svm_clf.fit(X_train, y_train)

SVC(kernel='linear')

In [169]:
y_pred = svm_clf.predict(X_test)

report = classification_report(y_test, y_pred)

print("Classification Report:")
print(report)

Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.97      0.88        63
           1       0.90      0.55      0.68        33

    accuracy                           0.82        96
   macro avg       0.85      0.76      0.78        96
weighted avg       0.84      0.82      0.81        96




Following are the keypoints:

Feature Representation:
------------------------
Naive Bayes relies on frequency-based features, which are counts of occurrences of words or other features in the dataset. This approach assumes that the frequency of features contributes to classification.
SVM with a linear kernel, on the other hand, does not make specific assumptions about feature distributions. It aims to find a hyperplane that separates the classes in the feature space.

Effect of POS Tagging:
------------------------
POS tagging introduces additional information about the linguistic structure of the text data. While this information may be beneficial for some classifiers, such as SVM, it may not always improve the performance of Naive Bayes, especially if it introduces inaccuracies.
In my case, POS tagging reduced the accuracy of Naive Bayes by 1%, indicating that the additional information introduced by POS tagging may have led to inaccuracies in the feature representation or the classification process.
However, for SVM with a linear kernel, POS tagging increased the accuracy by 2%, suggesting that the additional information provided by POS tagging improved the separability of the classes in the feature space, leading to better classification performance.

Linear Separability:
---------------------
The improvement in SVM performance with a linear kernel suggests that there may be a linear separation between the classes in the feature space. This means that the classes can be effectively separated by a linear decision boundary, which SVM with a linear kernel can exploit.
Based on these observations, it appears that the combination of POS tagging and SVM with a linear kernel was more effective for my classification task compared to Naive Bayes with frequency-based features. This highlights the importance of considering the characteristics of the data and the suitability of different classifiers and feature representations for the task at hand.

In [150]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

# Sample documents
corpus = [
    "This is the first document.",
    "This document is the second document.",
    "And this is the third one.",
    "Is this the first document?"
]

# Initialize CountVectorizer
vectorizer = CountVectorizer()

# Fit and transform the corpus
X = vectorizer.fit_transform(corpus)

# Convert the result to a pandas DataFrame
count_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

# Display the DataFrame
print(count_df)

   and  document  first  is  one  second  the  third  this
0    0         1      1   1    0       0    1      0     1
1    0         2      0   1    0       1    1      0     1
2    1         0      0   1    1       0    1      1     1
3    0         1      1   1    0       0    1      0     1
